In [2]:
import torch
import time
from loguru import logger

In [3]:
torch.__version__

'2.4.1'

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
logger.info(f"Using {device} device")

2024-09-06 23:15:37.260 | INFO     | __main__:<module>:8 - Using mps device


In [5]:
# Create a simple tensor
x = torch.randn(5, 5, device=device)
y = torch.randn(5, 5, device=device)

# Perform a matrix multiplication
z = torch.matmul(x, y)

print("Matrix multiplication on MPS:")
print(z)

Matrix multiplication on MPS:
tensor([[-0.1635, -0.7482,  1.6124,  0.7250,  1.5496],
        [-0.2358, -3.1579,  1.2337,  1.7647,  1.3374],
        [ 1.3663, -0.9042, -0.4299,  0.0171, -2.9793],
        [-1.1360,  2.5955,  2.3696,  0.6012,  2.8251],
        [ 0.6192, -1.8638,  1.5357,  0.2832,  1.8710]], device='mps:0')


In [6]:
# Function to measure time taken for an operation
def measure_time(device, operation, description):
    start_time = time.time()
    operation()
    end_time = time.time()
    logger.info(f"{description} on {device}: {end_time - start_time:.6f} seconds")

In [7]:
size = (10000, 10000)

# Define the CPU device
device_cpu = torch.device("cpu")

# Define the MPS device
device_mps = torch.device("mps")

# Create random tensors
x_cpu = torch.randn(size, device=device_cpu)
y_cpu = torch.randn(size, device=device_cpu)

x_mps = x_cpu.to(device_mps)
y_mps = y_cpu.to(device_mps)

# Matrix Multiplication
measure_time(device_cpu, lambda: torch.matmul(x_cpu, y_cpu), "Matrix multiplication")
measure_time(device_mps, lambda: torch.matmul(x_mps, y_mps), "Matrix multiplication")

# Element-wise addition
measure_time(device_cpu, lambda: x_cpu + y_cpu, "Element-wise addition")
measure_time(device_mps, lambda: x_mps + y_mps, "Element-wise addition")

# Simple neural network forward pass
class SimpleNN(torch.nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = torch.nn.Linear(1000, 500)
        self.fc2 = torch.nn.Linear(500, 100)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x

model_cpu = SimpleNN().to(device_cpu)
model_mps = SimpleNN().to(device_mps)

input_cpu = torch.randn(1000, 1000, device=device_cpu)
input_mps = input_cpu.to(device_mps)

measure_time(device_cpu, lambda: model_cpu(input_cpu), "Neural network forward pass")
measure_time(device_mps, lambda: model_mps(input_mps), "Neural network forward pass")

2024-09-06 23:15:40.753 | INFO     | __main__:measure_time:6 - Matrix multiplication on cpu: 1.560292 seconds
2024-09-06 23:15:40.928 | INFO     | __main__:measure_time:6 - Matrix multiplication on mps: 0.172979 seconds
2024-09-06 23:15:40.985 | INFO     | __main__:measure_time:6 - Element-wise addition on cpu: 0.056860 seconds
2024-09-06 23:15:40.991 | INFO     | __main__:measure_time:6 - Element-wise addition on mps: 0.004904 seconds
2024-09-06 23:15:41.603 | INFO     | __main__:measure_time:6 - Neural network forward pass on cpu: 0.007262 seconds
2024-09-06 23:15:41.618 | INFO     | __main__:measure_time:6 - Neural network forward pass on mps: 0.014297 seconds
